# Spark best practices
source : https://robertovitillo.com/2015/06/30/spark-best-practices/

In [1]:
rdd = sc.textFile('derby.log') \
        .flatMap(lambda line: line.split()) \
        .map(lambda word: (word, 1)) \
        .reduceByKey(lambda x,y: x+y, 3) \
        .collect()

## Spark UI

http://localhost:4040
## Use the right level of parallelism
In general, 2-3 task per CPU core in your cluster are recommended.
## Reduce working set size
be careful with operations resulting too large working set for the following tasks.  
Spark's shuffle operations(**sortByKey**, **groupByKey**, **reduceByKey**, **join**, etc) build a hash table within each task to perform the grouping, which can often be large.
## Avoid groupByKey for associative operations
both **reduceByKey** and **groupByKey** can be used for the same purposes but **reduceByKey** works much better on a large dataset.

## Aviod reduceByKey when the input and output value types are different

In [2]:
rdd = sc.textFile('derby.log') \
        .flatMap(lambda line: line.lower().split()) \
        .filter(lambda x: len(x) > 1)

In [3]:
# not good
rdd.map(lambda p: (p[0], {p[1]})) \
   .reduceByKey(lambda x,y: x|y) \
   .collect()

[(u'a', {u'8', u'p'}),
 (u'c', {u'l', u'o', u's'}),
 (u'i', {u'n'}),
 (u'm', {u'a'}),
 (u'-', {u'-'}),
 (u'3', {u'0'}),
 (u'o', {u'n', u'r', u's'}),
 (u'1', {u'0'}),
 (u's', {u'o', u't'}),
 (u'u', {u's'}),
 (u'/', {u'u'}),
 (u'w', {u'i'}),
 (u'b', {u'o'}),
 (u'd', {u'a', u'e', u'i'}),
 (u'f', {u'i', u'o', u'r'}),
 (u'(', {u'1'}),
 (u'j', {u'a'}),
 (u'l', {u'o'}),
 (u'0', {u'2'}),
 (u'2', {u'0'}),
 (u't', {u'h'}),
 (u'v', {u'e'})]

In [4]:
# better way
def seq_op(xs, x):
    xs.add(x)
    return xs

def comb_op(xs, ys):
    return xs | ys

rdd.map(lambda p: (p[0], p[1])).aggregateByKey(set(), seq_op, comb_op).collect()

[(u'a', {u'8', u'p'}),
 (u'c', {u'l', u'o', u's'}),
 (u'i', {u'n'}),
 (u'm', {u'a'}),
 (u'-', {u'-'}),
 (u'3', {u'0'}),
 (u'o', {u'n', u'r', u's'}),
 (u'1', {u'0'}),
 (u's', {u'o', u't'}),
 (u'u', {u's'}),
 (u'/', {u'u'}),
 (u'w', {u'i'}),
 (u'b', {u'o'}),
 (u'd', {u'a', u'e', u'i'}),
 (u'f', {u'i', u'o', u'r'}),
 (u'(', {u'1'}),
 (u'j', {u'a'}),
 (u'l', {u'o'}),
 (u'0', {u'2'}),
 (u'2', {u'0'}),
 (u't', {u'h'}),
 (u'v', {u'e'})]

## Avoid the flatMap-join-groupBy pattern
use **cogroup** instead
## Python memory overhead
set **spark.executor.memory** option
## Use broadcast variables
## Cache judiciously
## Don't collect large RDDs
use **take** or **takeSample**
## Minimize amount of data shuffled
it involves disk I/O, data serialization, network I/O and temporary storage.
## Know the standard library
## Use dataframes